In [13]:
#%%time
from setup import *

get_cliques = False
printout = False
np.random.seed(131)
num_verts = 20
ramsey = [3,10]
num_colorings = 9
num_steps = 240
beta = .5
exec(open("ramsey_parallel_new.py").read())

#colorings[0], colorings[-1] = colorings[-1], colorings[0]
#colorings = np.roll(colorings,3)

problem_counts_current = count_problems(colorings,printout)

problem_counts_proposed = copy(problem_counts_current)
problem_counts_best = copy(problem_counts_current.copy())
colorings_best = copy(colorings.copy())
best_step = np.zeros(num_colorings).astype('uint64')

def print_coloring(idx=None):
    df = pd.DataFrame(colorings)
    df.columns = [tuple(e) for e in edge_idx_to_pair]
    if idx is not None:
        df = df.loc[idx]    
    display(df)

def print_status(best_step,problem_counts_best):
    df = pd.DataFrame()
    df['best step'] = best_step
    df['problems'] = problem_counts_best
    df = df.T
    idx = np.argmin(problem_counts_best)#.argmin()
    s = 'best='+str(idx)
    df[s] = df[idx]
    display(df)
    

edge_color_old = np.zeros(num_colorings).astype('uint16')
for step in range(1,num_steps+1):    
    #print_coloring()
    found = (problem_counts_current == 0)
    if np.any(found):
        found = found.nonzero()
        print("FOUND AT LEAST ONE WITH NO PROBLEMS: %s"%found)
        print_status(best_step,problem_counts_best)
        print_coloring(found[0])
        break
    edge_idx = np.random.randint(0, tot_edges, size=num_colorings).astype('uint16')
    color_delta = np.random.randint(1, num_colors, size=num_colorings).astype('uint16')
    for clr in range(num_colorings):
        e = edge_idx[clr]
        edge_color_old[clr] = copy(colorings[clr,e])
        colorings[clr,e] = (colorings[clr,e] + color_delta[clr]) % num_colors   
        
    
    problem_counts_gpu *= 0
    #problems_gpu *= 0
    #problem_counts_gpu.set(np.zeros(num_colorings,dtype='uint32'))    
    #Z = np.zeros([num_colorings,tot_cliques],dtype='uint16')
    #problems_gpu.set(Z)
    problem_counts_proposed = count_problems(colorings,printout)
    problem_diff = problem_counts_current.astype('int16') - problem_counts_proposed.astype('int16')
    #print(problem_diff)
    for clr in range(num_colorings):
        if(problem_diff[clr] >= 0):
            problem_counts_current[clr] = copy(problem_counts_proposed[clr])
            if(problem_counts_best[clr] > problem_counts_proposed[clr]):
                problem_counts_best[clr] = copy(problem_counts_proposed[clr])
                colorings_best[clr] = copy(colorings[clr])
                best_step[clr] = step
        else:
            accept = np.exp(beta * problem_diff[clr])
            r = np.random.random()
            if r <= accept:
                problem_counts_current[clr] = copy(problem_counts_proposed[clr])
            else:
                #print("reject")
                colorings[clr,e] = copy(edge_color_old[clr])
#     print(problem_counts_current)
#     print(problem_counts_proposed)
#     print(problem_diff)
    if(step % 20 == 0):
        print("step %u"%step)
        print_status(best_step,problem_counts_best)

step 20


,0,1,2,3,4,5,6,7,8,best=0
best step,17,19,2,16,16,19,20,14,18,17
problems,73,119,123,147,119,129,165,196,73,73


step 40


,0,1,2,3,4,5,6,7,8,best=8
best step,17,34,2,40,16,40,37,40,36,36
problems,73,91,123,135,119,120,138,164,59,59


step 60


...

,0,1,2,3,4,5,6,7,8,best=8
best step,17,34,90,182,140,107,86,165,107,107
problems,73,91,102,126,104,78,95,116,31,31


step 220


,0,1,2,3,4,5,6,7,8,best=8
best step,17,34,90,214,140,107,217,219,219,219
problems,73,91,102,100,104,78,86,90,26,26


step 240


,0,1,2,3,4,5,6,7,8,best=8
best step,17,34,234,238,234,107,217,219,240,240
problems,73,91,98,74,90,78,86,90,24,24


In [18]:
print(problems_gpu.shape)
print(num_colorings*tot_cliques)
colorings[-2]
print(problems_current.shape)

(47589376,)
47589376
(256, 185896)


In [36]:
q = colorings[[-1]]
q.shape
print(colorings == q)
tot_verts
tot_edges

[[False False False ...,  True False False]
 [False False False ..., False  True False]
 [ True  True False ...,  True  True False]
 ..., 
 [ True False  True ..., False False  True]
 [ True False False ..., False  True  True]
 [ True  True  True ...,  True  True  True]]


190

In [25]:
for k,edge in enumerate(edges):
    edge_idx = edge_pair_to_idx[tuple(edge)]
    c.append(colorings[0,edge_idx])
c

/home/particle/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


[0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1]

In [37]:
from setup import *
from scipy.special import binom

num_colorings = 1

ramsey = np.asarray(ramsey)
ramsey.sort()
d_colorings = q
tot_verts = num_verts
tot_edges = int(binom(tot_verts,2))

edge_idx_to_pair = np.array(list(it.combinations(range(tot_verts),2)))
edge_pair_to_idx = np.zeros([tot_verts,tot_verts])
for (idx,e) in enumerate(edge_idx_to_pair):
    edge_pair_to_idx[tuple(e)] = idx
edge_pair_to_idx += edge_pair_to_idx.T
np.fill_diagonal(edge_pair_to_idx,tot_edges)


probs = np.zeros(num_colors,dtype='int')
for i, ram in enumerate(ramsey):
    cliques = it.combinations(range(num_verts),ram)
    for j,v in enumerate(cliques):        
        edges = list(it.combinations(v,2))        
        c = np.zeros(len(edges))
        edge_idx = np.zeros_like(c)
        for k,edge in enumerate(edges):
            edge_idx[k] = edge_pair_to_idx[tuple(edge)]
            #c.append(colorings[0,edge_idx])
            c[k] = d_colorings[0,edge_idx[k]]
        
        #c = np.asarray(c)
        if(np.all(c == i)):
            probs[i] += 1
            print("clr%u  clique%u  verts%s  edges%s  colors%s  %s"%(i,j,v,edge_idx,c, np.all(c == i)))
print(probs)

/home/particle/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


clr0  clique35  verts(0, 3, 4)  edges[  2.   3.  54.]  colors[ 0.  0.  0.]  True
clr0  clique59  verts(0, 4, 13)  edges[  3.  12.  78.]  colors[ 0.  0.  0.]  True
clr0  clique62  verts(0, 4, 16)  edges[  3.  15.  81.]  colors[ 0.  0.  0.]  True
clr0  clique180  verts(1, 2, 12)  edges[ 19.  29.  46.]  colors[ 0.  0.  0.]  True
clr0  clique258  verts(1, 8, 9)  edges[  25.   26.  124.]  colors[ 0.  0.  0.]  True
clr0  clique479  verts(3, 5, 10)  edges[ 55.  60.  89.]  colors[ 0.  0.  0.]  True
clr0  clique626  verts(4, 8, 16)  edges[  73.   81.  131.]  colors[ 0.  0.  0.]  True
clr0  clique921  verts(8, 9, 11)  edges[ 124.  126.  136.]  colors[ 0.  0.  0.]  True
clr0  clique926  verts(8, 9, 16)  edges[ 124.  131.  141.]  colors[ 0.  0.  0.]  True
clr0  clique1022  verts(10, 11, 14)  edges[ 145.  148.  156.]  colors[ 0.  0.  0.]  True
[10  0]


In [1]:
problems.shape
num_cliques_cum[-1]
tot_cliques
print(np.argwhere(problems_gpu[:tot_cliques]>0))

NameError: name 'problems' is not defined

In [49]:
problem_counts_current
(problems[0]>0).nonzero()

(array([], dtype=int64),)

In [18]:
problem_counts_best

array([12, 11,  9,  6, 10,  8,  8, 10,  8,  7, 10, 10, 11, 12,  8, 13,  8,
       11, 14,  8, 13, 10,  9,  7,  8,  8,  9, 11,  8,  9,  9, 12, 11,  6,
        9,  6,  8,  6, 11,  6,  9,  8, 12, 11,  6,  6, 12,  6,  7, 12,  8,
       10, 10, 13, 11,  8, 10,  8,  8, 15, 10,  9, 11, 13, 11,  9, 12,  8,
        6,  8,  9, 10,  7, 10,  6,  8,  8,  8, 10, 12,  9, 11,  8, 10, 10,
       10,  8,  9, 11,  4, 10, 10, 11,  9, 13,  7, 12, 11,  6,  6, 10, 11,
        7,  7,  8, 12, 10, 11,  9,  7, 13, 11,  8, 10, 10,  8,  8,  9,  7,
       10,  9,  7, 10, 15, 11, 12, 11,  7,  7,  9, 11,  7, 10, 12,  7,  6,
       12,  9,  8, 12,  6,  8,  9, 10,  9, 11,  9, 11, 10,  9,  7,  6,  9,
       10, 10,  9, 13, 10,  9,  8, 14, 12,  7,  8,  5, 12,  8, 10,  8, 13,
       10, 10, 14, 10, 15,  9, 12, 12,  8,  9,  3,  9,  9,  9,  8,  8, 11,
        8,  8,  8,  6, 15,  8,  8, 12, 10,  7,  9, 10, 11,  9, 12,  8,  9,
        7,  8, 12,  9,  9,  8,  7,  8, 12, 10,  7, 10, 10, 11,  8,  9, 17,
        8,  8,  6, 11, 13

In [28]:
c = colorings_best[-1]
c = np.tile(c,[num_colorings,1])

In [3]:
        current_edge = edge_idx[coloring]
        edge_color_old[coloring] = coloring_cpu[coloring][current_edge]
        coloring_cpu[coloring][current_edge] = (edge_color_old[coloring] + color_delta[coloring])%num_colors
        if(list(coloring_cpu[coloring]).count(green) < min_green):
            coloring_cpu[coloring] = increase_green_edges(coloring_cpu[coloring])    
    coloring_gpu.set(coloring_cpu)

    num_problems_proposed = find_problems_cuda(coloring_gpu)
    num_problems_gen = find_problems_gen(coloring_gpu)
    if np.array_equal(num_problems_proposed, num_problems_gen) == False:
        print(np.sum(num_problems_current == num_problems_gen))
        raise Exception("They don't agree")
    else:
        print("They do agree!")

    #num_problems_proposed = find_problems_cuda(coloring_gpu)
    num_problems_diff = num_problems_current - num_problems_proposed
    #print("current:",num_problems_current,"proposed", num_problems_proposed)
    for coloring in range(num_colors):
        current_edge = edge_idx[coloring]
        if num_problems_diff[coloring] >= 0:
            #print("Proposed is better.  Accepting.")            
            num_problems_current[coloring] = num_problems_proposed[coloring]
            #Problems_current = Problems_proposed.copy()
            #if num_problems_proposed[coloring] < num_problems_best:
            #    step_best = step
            #    coloring_best = coloring_cpu[coloring].copy()
            #    num_problems_best = num_problems_proposed
            #    #Problems_best = Problems_proposed.copy()
            #    print_status()
        else:            
            accept = np.exp(beta * num_problems_diff[coloring])            
            r = np.random.random()
            #print("Proposed is worse.") # But I will accept it anyway if I draw a number less than %.3f.  I drew %.3f." % (accept,r))            
            if r <= accept:            
                #print("So I accept the move even though it is worse.")                
                num_problems_current[coloring] = num_problems_proposed[coloring]
                #Problems_current = Problems_proposed.copy()
            else:                
                #print("So I reject.")
                coloring_cpu[coloring][current_edge] = edge_color_old[coloring]
    if np.min(num_problems_current)< num_problems_best:
        step_best = step
        best_coloring_id = np.argmin(num_problems_current)
        coloring_best = coloring_cpu[best_coloring_id].copy()
        num_problems_best = num_problems_current[best_coloring_id]
        print_status()
    step += 1
    loop_step += 1
    if(loop_step >= loop_length):
        loops_done += 1
        loop_step = 0
        print_status()
        compute_time = (datetime.datetime.now() - start_compute).seconds
        steps_done = loops_done*loop_length1+Ramsey[1]
        rate = steps_done*num_colorings / compute_time
        job_time = (num_steps-steps_done)/rate
        m, s = divmod(job_time,60)
        h, m = divmod(m,60)
        d, h = divmod(h,24)
        y, d = divmod(d,365)
        print("At %.0f colorings/second, it'll take me %d years %d days %d hours %d minutes and %d seconds to complete the remaining steps."%
              (rate,y,d,h,m,s))

print("FINISHED!!")


problem_counts = count_problems(colorings)

print(problem_counts)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.48 µs
[20  4 13  2 19 38 16 15 23 15 22 11 12 30 21 17]


In [1]:
from setup import *

num_verts = 10
ramsey = [3,5]
num_colorings = 1

num_edges = int(num_verts*(num_verts-1)/2)
num_colors = len(ramsey)

colorings = np.random.randint(0, num_colors, size=[num_colorings, num_edges], dtype = 'uint16')
exec(open("ramsey_parallel.py").read())



problems = count_problems_gpu(colorings)
print(problems)

ValueError: must specify block size

In [ ]:
colorings = np.random.randint

In [ ]:
colorings = np.random

In [ ]:
colorings = np.random.randomint

In [ ]:
colorings = np.randomint

## Goal for next update:
1. Adapt the Ramsey_parallel_multi_coloring so that the threads generate their indices. 

## Idea behind this update:
Compared to the amount of space taken up by the subgraph index array, a coloring takes up minimal space. Also, for R(4,6) and R(5,5), we have plenty of space left on the GPU's global array after sending down the complete subgraph index array. So, we should be able to send down a bunch of colorings in order to perform many independent Markov searches without having to adapt our code significantly. 

## NVIDIA DOC:
http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#maximize-instruction-throughput

LIMITS FOR THE DATA TYPES:

https://wiki.tiker.net/PyCuda/FrequentlyAskedQuestions?action=fullsearch&context=180&value=data+type&titlesearch=Titles#How_do_I_specify_the_correct_types_when_calling_and_preparing_PyCUDA_functions.3F

#int8 	Byte (-128 to 127)
#int16 	Integer (-32768 to 32767)
#int32 	Integer (-2147483648 to 2147483647)
#int64 	Integer (-9223372036854775808 to 9223372036854775807)
#uint8 	Unsigned integer (0 to 255)
#uint16 	Unsigned integer (0 to 65535)
#uint32 	Unsigned integer (0 to 4294967295)
#uint64 	Unsigned integer (0 to 18446744073709551615)

CONVERSION USING NUMPY:

(unsigned) char = numpy.(u)int8

(unsigned) short = numpy.(u)int16

(unsigned) int = numpy.(u)int32

(unsigned) long = numpy.(u)int64 (only 64-bit)

floats = numpy.float32

double = numpy.float64

all pointers ( e.g int *, float ***, anything at all) should be numpy.intp.

In [31]:
from setup import *
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import random

def main_random(Ramsey, num_vertices, num_steps, num_colorings = 5, beta=2):
    start_time = datetime.datetime.now()
    print()

    def choose(n,k):
        try: 
            return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))
        except:
            return 0
    #The theory for this function comes from the Lesser Paper. Theorem 7 there provides very useful lower-bounds 
    #for the number of red edges found 
    def min_green_edges(Ramsey, num_vertices):
        k = Ramsey[1]
        if len(Ramsey) != 2 or Ramsey[0] != 3:
            return(0)
        elif num_vertices <=2*k:
            return(num_vertices-k)
        elif num_vertices <= 5*k/2:
            return(3*num_vertices - 5*k)
        else:
            return(5*num_vertices - 10*k)
    Ramsey = np.sort(Ramsey)
    min_green = min_green_edges(Ramsey, num_vertices)
    num_colors = len(Ramsey)
    Colors = np.arange(num_colors)
    Vertices = np.arange(num_vertices)
    Edges = list(it.combinations(Vertices,2))
    #reverse lookup for edges below.  Eg if slot 3 above contains edge (2,5), the the dict below has entry (2,5):3
    Edges_idx = dict((edge, idx) for idx,edge in enumerate(Edges)) 
    num_edges = len(Edges)

    threads_per_block = 32
    vertices_per_clique = Ramsey
    edges_per_clique = np.array([choose(v,2) for v in vertices_per_clique])
    cliques_per_color = np.asarray([choose(num_vertices,v) for v in vertices_per_clique])
    blocks_per_color = np.ceil(cliques_per_color / threads_per_block).astype('uint32')
    num_blocks = blocks_per_color.sum()
    cliques_per_block = np.ceil(cliques_per_color / blocks_per_color).astype('uint32')
    #The objects below tells each block which color and how many cliques/edges it will monitor.
    #Note each vector is repetitive.  If color 0 gets 7 blocks, the first 7 entries will be the same
    block_color = np.repeat(Colors,blocks_per_color).astype('uint32')
    block_num_cliques = np.repeat(cliques_per_block,blocks_per_color).astype('uint32')
    block_edges_per_clique = np.repeat(edges_per_clique,blocks_per_color).astype('uint32')
    num_threads_per_coloring = num_blocks*threads_per_block
    
    assign_Blocks_to_Cliques = np.full([num_blocks,cliques_per_block.max(),edges_per_clique.max()],
                                       fill_value=num_edges, dtype='uint32')

    #Counters that that tracks the next open block and thread on each block
    next_open_block = 0    
    next_open_thread = np.zeros(num_blocks,dtype='int')
    for color, clique_size in enumerate(Ramsey):
        #Creates a generator to produce all cliques (the list of vertices).
        Cliques = it.combinations(Vertices,clique_size)
        #Makes the vector [0,1,2,...,num_blocks-1,0,1,2,...,num_blocks-1,....] of length num_cliques
        assign_Cliques_to_Blocks = np.arange(cliques_per_color[color]) % blocks_per_color[color]
        #randomizes assignment, but maintains clique counts
        np.random.shuffle(assign_Cliques_to_Blocks)
        #Starts at next open block
        assign_Cliques_to_Blocks += next_open_block
        
        for clique_Vertices, block in zip(Cliques,assign_Cliques_to_Blocks):
            #Gets the list of edges in this clique
            clique_Edges = list(it.combinations(clique_Vertices,2))
            #Converts it to edge_idx
            clique_Edges_idx = [Edges_idx[edge] for edge in clique_Edges]
            #Writes it to the correct block and next open thread on that block
            assign_Blocks_to_Cliques[block,next_open_thread[block],:edges_per_clique[color]] = clique_Edges_idx            
            next_open_thread[block] += 1
        next_open_block += blocks_per_color[color]

    #Code below sets up the GPU checker
    block_color_gpu = gpuarray.to_gpu(block_color)
    #block_num_cliques = gpuarray.to_gpu(block_num_cliques)
    block_edges_per_clique_gpu = gpuarray.to_gpu(block_edges_per_clique)
    assign_Blocks_to_Cliques_gpu = gpuarray.to_gpu(assign_Blocks_to_Cliques)    

    kernel_code ="""
    #include <stdio.h>
    __global__ void find_problems(int *block_color, int *edges_per_clique, int *edges, int *coloring, int *Problems, int edges_per_thread)//, int cliques)
    {
        __shared__ int shared_coloring[shared_size];
        //This marches first down the x-dimension, then the y-dimension and finally the z-dimension. I'm not positive that
        //this is the best way to allocate the blocks but it is simple and easy to implement. 
        //Also, for the sake of simplicity, I've made the blocks only 1-D.
        int Id = threadIdx.x + blockDim.x*blockIdx.x + blockDim.x*gridDim.x*blockIdx.y + blockDim.x*gridDim.x*blockDim.y*gridDim.y*blockIdx.z;
        
        //If Id < num_threads_per_coloring, it is a thread that deals with the first coloring. 
        //If num_threads_per_coloring =< Id < 2*num_threads_per_coloring, then it deals with the second coloring and etc.
        int coloring_Id = (int) Id/num_threads_per_coloring;
        
        //Also walks down x, y and z dimensions respectively.
        int blockid = blockIdx.x + gridDim.x*blockIdx.y + gridDim.x*gridDim.y*blockIdx.z;
        
        //These next two numbers are how the threads navigate the assignment array.
        int BlockId = blockid - num_blocks_per_coloring*coloring_Id;
        int CliqueId = Id - num_threads_per_coloring*coloring_Id;
        //color that the thread is looking for
        int color = block_color[BlockId];
        if(threadIdx.x < shared_size)
        {
            shared_coloring[threadIdx.x] = coloring[coloring_Id*(shared_size-1) + threadIdx.x];
        }
        //if(clique_idx < cliques)
        //{
            //This weird way of starting and stopping allows us to traverse the 3D-array of assign_Blocks_to_Cliques easily
            int start = CliqueId*edges_per_thread;
            int end = start + edges_per_clique[BlockId];
            int e = start;
            while((e < end) && (shared_coloring[edges[e]] == color))
            {
                e++;
            }
            //Problems[clique_idx] = (e >= end) ? 1 : 0;
            if(e>=end)
            {
                atomicAdd(&Problems[coloring_Id], 1);
            }
        //}
    }
    """
    #The point of this is to get an choose_table where choose_table[k*len_choose + j] = k choose j. I then pass it down
    #to the GPU as a look up table
    choose_table = np.array(pd.DataFrame([[choose(j,i) for i in range(num_vertices+1)] for j in range(num_vertices+1)]))
    len_choose = choose_table.shape[1]
    choose_table = choose_table.ravel()
    gpu_choose = gpuarray.to_gpu(choose_table)
    
    gen_code ="""
    #include <stdio.h>
    __global__ void find_problems(int *coloring, int *choose_table, int *Problems)
    {
        //Each block will have the same index of clique to check but different coloring to check.
        //Also, we need to allocate enough room for either the green or the blue clique, hence the blue_size and blue_length.
        __shared__ int vertices[(blue_size  +1)];
        __shared__ int edges[blue_length+1];
        
        int e = 0; int l = 0;
        int clique_id = blockIdx.x + blockIdx.y*gridDim.x + blockIdx.z*gridDim.x*gridDim.y;
        int coloring_id = threadIdx.x;
        //split_point is the number of green cliques. So, if clique_id below, it checks a green clique and, if above, a blue.
        if(clique_id < split_point)
        {
            //Note, examine the gen_regex_dictionary below to see what a lot of these actually are. Most of the names are
            //intuitive though.
            int color = green; int N = clique_id; l = green_length;  
            
            //This is an algorithm to convert a natural number into a l-combination or the vertex index for the 
            //corresponding clique. 
            for(int i = 0; i < green_size; i++)
            {
                int k = green_size - i;
                int j = 0;
                while(choose_table[j*len_choose + k] <= N)
                {
                    j++;
                }
                vertices[k] = j;
                if(clique_id == 0 && coloring_id == 0)
                {
                    Problems[i] += k;
                    Problems[i+green_size] += N;
                    Problems[i+2*green_size] += j;
                }
                N -= choose_table[j*len_choose+k];
            }
            //This saves the last chunk of array as a -1 to make sure the later problem-checking algorithm knows to stop. 
            for(int i = green_size; i < blue_size; i++)    
            {     
                vertices[i] = -1;     
            }
            //This converts vertices into edges for the coloring vector and is the reverse process of the previous algorithm. 
            //It takes the first vertex, pairs with every other vertex after it and finding the edge corresponding. Then, it
            //repeats the process for the second vertex and the ones after it and so on. 
            for(int i = 0; i < green_size; i++)
            {
                int start = 0; int temp = i; 
                //The first vertex will take red_size - 1 spots, the second red_size - 2 and etc. This finds the starting point
                //for each. So, if second vertex, i = 1 and start = red_size - 1. If third, i = 2 
                while(temp > 0)
                {
                    start += green_size - temp; 
                    temp -= 1;
                }
                for(int j = i+1; j < green_size; j++)
                {
                    edges[start + j] = choose_table[(vertices[j]-1)*len_choose + 2] + vertices[i];
                }
            }
            /*if(clique_id == 0)
            {
                atomicAdd(&Problems[coloring_id], e);
            }*/
            //This will actually check the edges.
            while((e < l) && (coloring[coloring_id*(num_edges_per_coloring+1)]==color))// + edges[e]
            {
                e++;
            }
        }
        else
        {
            int color = blue; int N = clique_id - split_point; l = blue_length;
            for(int i = 0; i < blue_size; i++)
            {
                int k = blue_size - i;
                int j = 0;
                while(choose_table[(j-1)*len_choose + k] <= N)
                {
                    j++;
                }
                //This will save the correct number for the current spot and -1 for the next. It will also make the last
                //spot on the vertices array -1.
                vertices[k] = j; //vertices[i+1] = -1;
                N -= choose_table[(j-1)*len_choose+k];
            }
            for(int i = 0; i < blue_size; i++)
            {
                int start = 0; int temp = i; 
                while(temp > 0)
                {
                    start += blue_size - temp; 
                    temp -= 1;
                }
                for(int j = i+1; j < blue_size; j++)
                {
                    edges[start + j] = choose_table[(vertices[j]-1)*len_choose + 2] + vertices[i];
                }
            }
            //This will actually check the edges.
            
            while((e < l) && (coloring[coloring_id*(num_edges_per_coloring+1) + edges[e]]== color) )//
            {
                e++;
            }
        }
        if(e>=l)        
        {
            //atomicAdd(&Problems[coloring_id], 1);
        }
    }
    """
    regex_fixing_dictionary = {"shared_size":(num_edges+1), "num_threads_per_coloring":(num_threads_per_coloring), 
                              "num_blocks_per_coloring":num_blocks}
    
    gen_fixing_dictionary = {"blue_size":(Ramsey[1]), "green_size":(Ramsey[0]), "len_choose":(len_choose),
                             "split_point":cliques_per_color[0], "blue_length":(edges_per_clique[1]) ,
                             "green_length":(edges_per_clique[0]),"blue":(1), "green":(0),
                             "num_edges_per_coloring":(num_edges)}
    
    for key, val in regex_fixing_dictionary.items():
        kernel_code = kernel_code.replace(str(key),str(val))
    for key, val in gen_fixing_dictionary.items():
        gen_code = gen_code.replace(str(key),str(val))
    mod = SourceModule(kernel_code)
    G, B, edges_per_thread = assign_Blocks_to_Cliques_gpu.shape
    num_blocks_total = G*num_colorings
    #Creating a grid that is a cube. The easiest way to do it is to find smallest possible cube that fits it and just
    #allocate two many blocks. Later, I might experiment with other shapes to see if it affects it some.
    grid_dimensions = (int(num_blocks_total**(1/3)) + 1,int(num_blocks_total**(1/3)) + 1,int(num_blocks_total**(1/3)) + 1)
    block_dimensions = (B,1,1)
    edges_per_thread = np.uint32(edges_per_thread)
    print("#blocks = gridDim.x = %d, cliques per block = threads per block = blockDim.x = %d, edges per thread = %d"%(G,B,edges_per_thread))
    func = mod.get_function("find_problems")
    #Note, the current vesion has cuda add up the number of problems on a coloring all by itself and returns an array
    #where the first element is the number of problems in the first coloring, the second element is the number of problems
    #in the second coloring and etc.
    def find_problems_cuda(coloring_gpu, printout=False):
        Problems_gpu = gpuarray.to_gpu(np.zeros(num_colorings).astype("int32"))
        func(block_color_gpu, block_edges_per_clique_gpu, assign_Blocks_to_Cliques_gpu, coloring_gpu, Problems_gpu, edges_per_thread, block=block_dimensions, grid=grid_dimensions, shared=0)
        Problems_cpu = Problems_gpu.get().astype("int32")
        return Problems_cpu
    
    gen_mod = SourceModule(gen_code)
    gen_func = gen_mod.get_function("find_problems")
    
    def find_problems_gen(coloring_gpu, printout = False):
        gen_Problems= gpuarray.to_gpu(np.zeros(num_colorings).astype("int32"))
        gen_func(coloring_gpu, gpu_choose, gen_Problems, block = block_dimensions, grid = grid_dimensions, shared = 0)
        print(gen_Problems)
        Problems_cpu = gen_Problems.get()#.astype("int32")
        return Problems_cpu

    def print_problems(Problems):        
        Z = pd.DataFrame(Problems.astype('uint32'))
        Z.insert(0,'problems',Z.sum(axis=1))
        Z.insert(0,'color',block_color)
        Z = Z.T
        problem_idx = np.any(Z,axis=-1)
        problem_idx[:2] = True
        display(Z.ix[problem_idx,:])

    def print_status():
        now = datetime.datetime.now()
        elapsed = now - start_time
        print("%d steps done in %s.  Best coloring so far was coloring %d in step %d with %d problems.  Time now %s."
                   %(step,str(elapsed).split('.')[0],best_coloring_id, step_best,num_problems_best,str(now).split('.')[0]))
    def increase_green_edges(coloring):
        while(list(coloring).count(green)<min_green):
            idx = [i for i in range(len(coloring)) if coloring[i]!=green]
            coloring[random.choice(idx)] = green
        return(coloring)

    #Initialize the Markov chain
    test = []
    coloring_cpu = np.array([np.random.choice(Colors, size=num_edges+1, replace=True).astype('uint32') for coloring in range(num_colorings)])
    np.random.shuffle(coloring_cpu)
    for coloring in range(num_colorings):
        coloring_cpu[coloring][num_edges] = num_colors
        test.append(coloring_cpu[coloring][0])
    best = coloring_cpu.copy()
    coloring_gpu = gpuarray.to_gpu(coloring_cpu.copy())

    num_problems_current = find_problems_cuda(coloring_gpu)
    num_problems_gen = find_problems_gen(coloring_gpu)
    if np.array_equal(num_problems_current, num_problems_gen) == False:
        print(num_problems_current)
        print(num_problems_gen)
        raise Exception("They don't agree")
    else:
        print("They do agree!")
    
    num_problems_proposed = num_problems_current    
    num_problems_best = np.min(num_problems_current)
    best_coloring_id = np.argmin(num_problems_current)
    coloring_best = coloring_cpu[best_coloring_id].copy()
    green = min(vertices_per_clique)
    step = 0
    step_best = step
    
    loop_length = 100000
    loop_step = 0
    loops_done = 0
    start_compute = datetime.datetime.now()
    for i in range(num_steps):
        if num_problems_best == 0:
            break
        #This creates a random edge to switch, a random amount to change it by and an array of each of the old edge
        #colors. 
        #coloring_cpu = np.array([np.random.choice(Colors, size=num_edges+1, replace=True).astype('uint32') for coloring in range(num_colorings)])
        edge_idx = np.array([np.random.randint(0,num_edges) for coloring in range(num_colorings)])
        color_delta = np.array([np.random.randint(1,num_colors) for coloring in range(num_colorings)])
        edge_color_old = np.ones(num_colorings)*-1
        #For each coloring, find edge to switch, amount to switch by, actually change it and then ensure it has 
        #enough red edges
        for coloring in range(num_colorings):
            current_edge = edge_idx[coloring]
            edge_color_old[coloring] = coloring_cpu[coloring][current_edge]
            coloring_cpu[coloring][current_edge] = (edge_color_old[coloring] + color_delta[coloring])%num_colors
            if(list(coloring_cpu[coloring]).count(green) < min_green):
                coloring_cpu[coloring] = increase_green_edges(coloring_cpu[coloring])    
        coloring_gpu.set(coloring_cpu)

        num_problems_proposed = find_problems_cuda(coloring_gpu)
        num_problems_gen = find_problems_gen(coloring_gpu)
        if np.array_equal(num_problems_proposed, num_problems_gen) == False:
            print(np.sum(num_problems_current == num_problems_gen))
            raise Exception("They don't agree")
        else:
            print("They do agree!")
            
        #num_problems_proposed = find_problems_cuda(coloring_gpu)
        num_problems_diff = num_problems_current - num_problems_proposed
        #print("current:",num_problems_current,"proposed", num_problems_proposed)
        for coloring in range(num_colors):
            current_edge = edge_idx[coloring]
            if num_problems_diff[coloring] >= 0:
                #print("Proposed is better.  Accepting.")            
                num_problems_current[coloring] = num_problems_proposed[coloring]
                #Problems_current = Problems_proposed.copy()
                #if num_problems_proposed[coloring] < num_problems_best:
                #    step_best = step
                #    coloring_best = coloring_cpu[coloring].copy()
                #    num_problems_best = num_problems_proposed
                #    #Problems_best = Problems_proposed.copy()
                #    print_status()
            else:            
                accept = np.exp(beta * num_problems_diff[coloring])            
                r = np.random.random()
                #print("Proposed is worse.") # But I will accept it anyway if I draw a number less than %.3f.  I drew %.3f." % (accept,r))            
                if r <= accept:            
                    #print("So I accept the move even though it is worse.")                
                    num_problems_current[coloring] = num_problems_proposed[coloring]
                    #Problems_current = Problems_proposed.copy()
                else:                
                    #print("So I reject.")
                    coloring_cpu[coloring][current_edge] = edge_color_old[coloring]
        if np.min(num_problems_current)< num_problems_best:
            step_best = step
            best_coloring_id = np.argmin(num_problems_current)
            coloring_best = coloring_cpu[best_coloring_id].copy()
            num_problems_best = num_problems_current[best_coloring_id]
            print_status()
        step += 1
        loop_step += 1
        if(loop_step >= loop_length):
            loops_done += 1
            loop_step = 0
            print_status()
            compute_time = (datetime.datetime.now() - start_compute).seconds
            steps_done = loops_done*loop_length1+Ramsey[1]
            rate = steps_done*num_colorings / compute_time
            job_time = (num_steps-steps_done)/rate
            m, s = divmod(job_time,60)
            h, m = divmod(m,60)
            d, h = divmod(h,24)
            y, d = divmod(d,365)
            print("At %.0f colorings/second, it'll take me %d years %d days %d hours %d minutes and %d seconds to complete the remaining steps."%
                  (rate,y,d,h,m,s))

    print("FINISHED!!")
    #coloring_cpu = coloring_best.copy()start 
    #coloring_gpu.set(coloring_best)
    #num_problems_best = find_problems_cuda(coloring_gpu)
    print()
    print_status()
    final_coloring = pd.DataFrame()
    final_coloring['edge'] = Edges
    final_coloring['color'] = coloring_best[:num_edges]
    display(final_coloring)
    return final_coloring

In [35]:
num_vertices = 6
def choose(n,k):
        if(n>=k): 
            return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))
        else:
            return 0
choose_table = np.array(pd.DataFrame([[choose(j,i) for i in range(num_vertices+1)] for j in range(num_vertices+1)]))
len_choose = choose_table.shape[1]
print(choose_table)
choose_table = choose_table.ravel()
print(choose(40,10))
for k in range(num_vertices):
    for m in range(num_vertices):
        print("%d choose %d is: %d"% (int(m), int(k), int(choose_table[(m)*len_choose + k])))

[[ 1  0  0  0  0  0  0]
 [ 1  1  0  0  0  0  0]
 [ 1  2  1  0  0  0  0]
 [ 1  3  3  1  0  0  0]
 [ 1  4  6  4  1  0  0]
 [ 1  5 10 10  5  1  0]
 [ 1  6 15 20 15  6  1]]
847660528
0 choose 0 is: 1
1 choose 0 is: 1
2 choose 0 is: 1
3 choose 0 is: 1
4 choose 0 is: 1
5 choose 0 is: 1
0 choose 1 is: 0
1 choose 1 is: 1
2 choose 1 is: 2
3 choose 1 is: 3
4 choose 1 is: 4
5 choose 1 is: 5
0 choose 2 is: 0
1 choose 2 is: 0
2 choose 2 is: 1
3 choose 2 is: 3
4 choose 2 is: 6
5 choose 2 is: 10
0 choose 3 is: 0
1 choose 3 is: 0
2 choose 3 is: 0
3 choose 3 is: 1
4 choose 3 is: 4
5 choose 3 is: 10
0 choose 4 is: 0
1 choose 4 is: 0
2 choose 4 is: 0
3 choose 4 is: 0
4 choose 4 is: 1
5 choose 4 is: 5
0 choose 5 is: 0
1 choose 5 is: 0
2 choose 5 is: 0
3 choose 5 is: 0
4 choose 5 is: 0
5 choose 5 is: 1


In [32]:
Ramsey = [3,5]
num_vertices = 13
num_colorings = 32 
num_steps = 3*10**1
import datetime 

#Ramsey = [3,3,4](assign_Blocks_to_Cliques.shape)
#num_vertices = 30
#num_steps = 1000000000
beta = 1
bill = main_random(Ramsey, num_vertices, num_steps, num_colorings, beta=beta)


#blocks = gridDim.x = 50, cliques per block = threads per block = blockDim.x = 32, edges per thread = 10
[  3   2   1   0 -78 -78  54   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[41 59 55 46 55 54 77 61 38 54 46 56 75 64 44 41 48 75 37 41 61 31 54 44 56
 43 33 47 38 39 45 54]
[  3   2   1   0 -78 -78  54   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


Exception: They don't agree

# DO NOT TOUCH WHAT IS BELOW. 

In [ ]:
from setup import *
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import random

def main_random(Ramsey, num_vertices, num_steps, num_colorings = 5, beta=2):
    start_time = datetime.datetime.now()
    print()

    def choose(n,k):
        return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))
    #The theory for this function comes from the Lesser Paper. Theorem 7 there provides very useful lower-bounds 
    #for the number of red edges found 
    def min_red_edges(Ramsey, num_vertices):
        sorted_Ramsey = np.sort(Ramsey)
        k = sorted_Ramsey[1]
        if len(sorted_Ramsey) != 2 or sorted_Ramsey[0] != 3:
            return(0)
        elif num_vertices <=2*k:
            return(num_vertices-k)
        elif num_vertices <= 5*k/2:
            return(3*num_vertices - 5*k)
        else:
            return(5*num_vertices - 10*k)
    min_red = min_red_edges(Ramsey, num_vertices)
    num_colors = len(Ramsey)
    Colors = np.arange(num_colors)
    Vertices = np.arange(num_vertices)
    Edges = list(it.combinations(Vertices,2))
    #reverse lookup for edges below.  Eg if slot 3 above contains edge (2,5), the the dict below has entry (2,5):3
    Edges_idx = dict((edge, idx) for idx,edge in enumerate(Edges)) 
    num_edges = len(Edges)

    threads_per_block = 1024
    vertices_per_clique = Ramsey
    edges_per_clique = np.array([choose(v,2) for v in vertices_per_clique])
    cliques_per_color = np.asarray([choose(num_vertices,v) for v in vertices_per_clique])
    blocks_per_color = np.ceil(cliques_per_color / threads_per_block).astype('uint32')
    num_blocks = blocks_per_color.sum()
    cliques_per_block = np.ceil(cliques_per_color / blocks_per_color).astype('uint32')
    #The objects below tells each block which color and how many cliques/edges it will monitor.
    #Note each vector is repetitive.  If color 0 gets 7 blocks, the first 7 entries will be the same
    block_color = np.repeat(Colors,blocks_per_color).astype('uint32')
    block_num_cliques = np.repeat(cliques_per_block,blocks_per_color).astype('uint32')
    block_edges_per_clique = np.repeat(edges_per_clique,blocks_per_color).astype('uint32')
    num_threads_per_coloring = num_blocks*threads_per_block
    
    assign_Blocks_to_Cliques = np.full([num_blocks,cliques_per_block.max(),edges_per_clique.max()],
                                       fill_value=num_edges, dtype='uint32')

    #Counters that that tracks the next open block and thread on each block
    next_open_block = 0    
    next_open_thread = np.zeros(num_blocks,dtype='int')
    for color, clique_size in enumerate(Ramsey):
        #Creates a generator to produce all cliques (the list of vertices).
        Cliques = it.combinations(Vertices,clique_size)
        #Makes the vector [0,1,2,...,num_blocks-1,0,1,2,...,num_blocks-1,....] of length num_cliques
        assign_Cliques_to_Blocks = np.arange(cliques_per_color[color]) % blocks_per_color[color]
        #randomizes assignment, but maintains clique counts
        np.random.shuffle(assign_Cliques_to_Blocks)
        #Starts at next open block
        assign_Cliques_to_Blocks += next_open_block
        
        for clique_Vertices, block in zip(Cliques,assign_Cliques_to_Blocks):
            #Gets the list of edges in this clique
            clique_Edges = list(it.combinations(clique_Vertices,2))
            #Converts it to edge_idx
            clique_Edges_idx = [Edges_idx[edge] for edge in clique_Edges]
            #Writes it to the correct block and next open thread on that block
            assign_Blocks_to_Cliques[block,next_open_thread[block],:edges_per_clique[color]] = clique_Edges_idx            
            next_open_thread[block] += 1
        next_open_block += blocks_per_color[color]

    #Code below sets up the GPU checker
    block_color_gpu = gpuarray.to_gpu(block_color)
    #block_num_cliques = gpuarray.to_gpu(block_num_cliques)
    block_edges_per_clique_gpu = gpuarray.to_gpu(block_edges_per_clique)
    assign_Blocks_to_Cliques_gpu = gpuarray.to_gpu(assign_Blocks_to_Cliques)    

    kernel_code ="""
    #include <stdio.h>
    __global__ void find_problems(int *block_color, int *edges_per_clique, int *edges, int *coloring, int *Problems, int edges_per_thread)//, int cliques)
    {
        __shared__ int shared_coloring[shared_size];
        //This marches first down the x-dimension, then the y-dimension and finally the z-dimension. I'm not positive that
        //this is the best way to allocate the blocks but it is simple and easy to implement. 
        //Also, for the sake of simplicity, I've made the blocks only 1-D.
        int Id = threadIdx.x + blockDim.x*blockIdx.x + blockDim.x*gridDim.x*blockIdx.y + blockDim.x*gridDim.x*blockDim.y*gridDim.y*blockIdx.z;
        
        //If Id < num_threads_per_coloring, it is a thread that deals with the first coloring. 
        //If num_threads_per_coloring =< Id < 2*num_threads_per_coloring, then it deals with the second coloring and etc.
        int coloring_Id = (int) Id/num_threads_per_coloring;
        
        //Also walks down x, y and z dimensions respectively.
        int blockid = blockIdx.x + gridDim.x*blockIdx.y + gridDim.x*gridDim.y*blockIdx.z;
        
        //These next two numbers are how the threads navigate the assignment array.
        int BlockId = blockid - num_blocks_per_coloring*coloring_Id;
        int CliqueId = Id - num_threads_per_coloring*coloring_Id;
        //color that the thread is looking for
        int color = block_color[BlockId];
        if(threadIdx.x < shared_size)
        {
            shared_coloring[threadIdx.x] = coloring[coloring_Id*(shared_size-1) + threadIdx.x];
        }
        //if(clique_idx < cliques)
        //{
            //This weird way of starting and stopping allows us to traverse the 3D-array of assign_Blocks_to_Cliques easily
            int start = CliqueId*edges_per_thread;
            int end = start + edges_per_clique[BlockId];
            int e = start;
            while((e < end) && (shared_coloring[edges[e]] == color))
            {
                e++;
            }
            //Problems[clique_idx] = (e >= end) ? 1 : 0;
            if(e>=end)
            {
                atomicAdd(&Problems[coloring_Id], 1);
            }
        //}
    }
    """
    
    regex_fixing_dictionary = {"shared_size":(num_edges+1), "num_threads_per_coloring":(num_threads_per_coloring), 
                              "num_blocks_per_coloring":num_blocks}

    for key, val in regex_fixing_dictionary.items():
        kernel_code = kernel_code.replace(str(key),str(val))
    mod = SourceModule(kernel_code)
    G, B, edges_per_thread = assign_Blocks_to_Cliques_gpu.shape
    num_blocks_total = G*num_colorings
    #Creating a grid that is a cube. The easiest way to do it is to find smallest possible cube that fits it and just
    #allocate two many blocks. Later, I might experiment with other shapes to see if it affects it some.
    grid_dimensions = (int(num_blocks_total**(1/3)) + 1,int(num_blocks_total**(1/3)) + 1,int(num_blocks_total**(1/3)) + 1)
    block_dimensions = (B,1,1)
    edges_per_thread = np.uint32(edges_per_thread)
    print("#blocks = gridDim.x = %d, cliques per block = threads per block = blockDim.x = %d, edges per thread = %d"%(G,B,edges_per_thread))
    func = mod.get_function("find_problems")
    #Note, the current vesion has cuda add up the number of problems on a coloring all by itself and returns an array
    #where the first element is the number of problems in the first coloring, the second element is the number of problems
    #in the second coloring and etc.
    def find_problems_cuda(coloring_gpu, printout=False):
        Problems_gpu = gpuarray.to_gpu(np.zeros(num_colorings).astype("int32"))
        func(block_color_gpu, block_edges_per_clique_gpu, assign_Blocks_to_Cliques_gpu, coloring_gpu, Problems_gpu, edges_per_thread, block=block_dimensions, grid=grid_dimensions, shared=0)
        Problems_cpu = Problems_gpu.get().astype("int32")
        if printout == True:
            print_problems(Problems_cpu)
        return Problems_cpu
#   Here the start of the Pandas code. Used to double check my find_problems_cuda

    #compare = np.full_like(assign_Blocks_to_Cliques, fill_value=num_colors)
    #print(assign_Blocks_to_Cliques.shape)
    #for block in range(num_blocks):
    #    compare[block,:,:block_edges_per_clique[block]] = block_color[block]
    #
    #def find_problems_pandas(coloring, printout=False):
    #    X = coloring[assign_Blocks_to_Cliques]
    #    Y = (X == compare)
    #    Problems = np.all(Y,axis=-1)
    #    if printout == True:
    #        print_problems(Problems)
    #    return Problems.sum().astype('int'), Problems

    def print_problems(Problems):        
        Z = pd.DataFrame(Problems.astype('uint32'))
        Z.insert(0,'problems',Z.sum(axis=1))
        Z.insert(0,'color',block_color)
        Z = Z.T
        problem_idx = np.any(Z,axis=-1)
        problem_idx[:2] = True
        display(Z.ix[problem_idx,:])

    def print_status():
        now = datetime.datetime.now()
        elapsed = now - start_time
        print("%d steps done in %s.  Best coloring so far was coloring %d in step %d with %d problems.  Time now %s."
                   %(step,str(elapsed).split('.')[0],best_coloring_id, step_best,num_problems_best,str(now).split('.')[0]))
    def increase_red_edges(coloring):
        while(list(coloring).count(red)<min_red):
            idx = [i for i in range(len(coloring)) if coloring[i]!=red]
            coloring[random.choice(idx)] = red
        return(coloring)
    #Initialize the Markov chain
    coloring_cpu = np.array([np.random.choice(Colors, size=num_edges+1, replace=True).astype('uint32') for coloring in range(num_colorings)])
    np.random.shuffle(coloring_cpu)
    for coloring in range(num_colorings):
        coloring_cpu[coloring][num_edges] = num_colors
    coloring_best = coloring_cpu.copy()
    coloring_gpu = gpuarray.to_gpu(coloring_cpu.copy())

    #num_problems_current, Problems_current = find_problems_pandas(coloring_cpu, printout=False)
    #num_problems_current, Problems_current = find_problems_cuda(coloring_gpu, get_from_gpu=True, printout=False)
    num_problems_current = find_problems_cuda(coloring_gpu)    
    num_problems_proposed = num_problems_current    
    num_problems_best = np.min(num_problems_current)
    best_coloring_id = np.argmin(num_problems_current)
    coloring_best = coloring_cpu[best_coloring_id].copy()
    red = min(vertices_per_clique)
    step = 0
    step_best = step
    
    loop_length = 100000
    loop_step = 0
    loops_done = 0
    start_compute = datetime.datetime.now()
    for i in range(num_steps):
        if num_problems_best == 0:
            break
        #This creates a random edge to switch, a random amount to change it by and an array of each of the old edge
        #colors. 
        #coloring_cpu = np.array([np.random.choice(Colors, size=num_edges+1, replace=True).astype('uint32') for coloring in range(num_colorings)])
        edge_idx = np.array([np.random.randint(0,num_edges) for coloring in range(num_colorings)])
        color_delta = np.array([np.random.randint(1,num_colors) for coloring in range(num_colorings)])
        edge_color_old = np.ones(num_colorings)*-1
        #For each coloring, find edge to switch, amount to switch by, actually change it and then ensure it has 
        #enough red edges
        for coloring in range(num_colorings):
            current_edge = edge_idx[coloring]
            edge_color_old[coloring] = coloring_cpu[coloring][current_edge]
            coloring_cpu[coloring][current_edge] = (edge_color_old[coloring] + color_delta[coloring])%num_colors
            if(list(coloring_cpu[coloring]).count(red) < min_red):
                coloring_cpu[coloring] = increase_red_edges(coloring_cpu[coloring])    
        coloring_gpu.set(coloring_cpu)

#         The code below check the pandas and cuda versions against each other.
#         It is commented out by default because it slows things down.
#         If you want to use it, you also need to uncomment several lines above to activate the pandas algorithm.

        #num_problems_proposed, Problems_proposed_pandas = find_problems_pandas(coloring_cpu)#, printout=True)
        #num_problems_proposed, Problems_proposed_cuda = find_problems_cuda(coloring_gpu, get_from_gpu=True, printout=False)
        #if np.all(Problems_proposed_pandas == Problems_proposed_cuda) == True:
        #    print("Pandas and Cuda agree!!")
        #else:
        #    raise Exception("Pandas and Cuda disagree :()")
            
        num_problems_proposed = find_problems_cuda(coloring_gpu)
        num_problems_diff = num_problems_current - num_problems_proposed
        #print("current:",num_problems_current,"proposed", num_problems_proposed)
        for coloring in range(num_colors):
            current_edge = edge_idx[coloring]
            if num_problems_diff[coloring] >= 0:
                #print("Proposed is better.  Accepting.")            
                num_problems_current[coloring] = num_problems_proposed[coloring]
                #Problems_current = Problems_proposed.copy()
                #if num_problems_proposed[coloring] < num_problems_best:
                #    step_best = step
                #    coloring_best = coloring_cpu[coloring].copy()
                #    num_problems_best = num_problems_proposed
                #    #Problems_best = Problems_proposed.copy()
                #    print_status()
            else:            
                accept = np.exp(beta * num_problems_diff[coloring])            
                r = np.random.random()
                #print("Proposed is worse.") # But I will accept it anyway if I draw a number less than %.3f.  I drew %.3f." % (accept,r))            
                if r <= accept:            
                    #print("So I accept the move even though it is worse.")                
                    num_problems_current[coloring] = num_problems_proposed[coloring]
                    #Problems_current = Problems_proposed.copy()
                else:                
                    #print("So I reject.")
                    coloring_cpu[coloring][current_edge] = edge_color_old[coloring]
        if np.min(num_problems_current)< num_problems_best:
            step_best = step
            best_coloring_id = np.argmin(num_problems_current)
            coloring_best = coloring_cpu[best_coloring_id].copy()
            num_problems_best = num_problems_current[best_coloring_id]
            print_status()
        step += 1
        loop_step += 1
        if(loop_step >= loop_length):
            loops_done += 1
            loop_step = 0
            print_status()
            compute_time = (datetime.datetime.now() - start_compute).seconds
            steps_done = loops_done*loop_length
            rate = steps_done*num_colorings / compute_time
            job_time = (num_steps-steps_done)/rate
            m, s = divmod(job_time,60)
            h, m = divmod(m,60)
            d, h = divmod(h,24)
            y, d = divmod(d,365)
            print("At %.0f colorings/second, it'll take me %d years %d days %d hours %d minutes and %d seconds to complete the remaining steps."%
                  (rate,y,d,h,m,s))

    print("FINISHED!!")
    #coloring_cpu = coloring_best.copy()
    #coloring_gpu.set(coloring_best)
    #num_problems_best = find_problems_cuda(coloring_gpu)
    print()
    print_status()
    final_coloring = pd.DataFrame()
    final_coloring['edge'] = Edges
    final_coloring['color'] = coloring_best[:num_edges]
    display(final_coloring)
    return final_coloring

# YOU MAY NOW EDIT CODE AGAIN. DO NOT TOUCH WHAT IS DIRECTLY ABOVE ME

In [ ]:
Ramsey = [4,6]
num_vertices = 35

num_steps = 326#50000#This takes just under 18 hours to run
import datetime 
#Ramsey = [3,3,4]
#num_vertices = 30
#num_steps = 1000000000

beta = 1
bill = main_random(Ramsey, num_vertices, num_steps, num_colorings=5,beta=beta)